### Notebook to run the complete eval of our method for Handmade questions

https://docs.google.com/spreadsheets/d/1xgkQt_1NiwXEZhZUvTypgecG5wMCl8_brc6w1bhaOtE/edit?gid=0#gid=0



In [1]:
# Optional : rm -r storage to remove storage


In [2]:
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
DEVICE = "cuda:0" #only runs fast on GPU

HF_CACHE = "/home/hice1/bzhou91/scratch/cache"
print("loading models")
BT_processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=HF_CACHE) #.to(DEVICE)
BT_model = BridgeTowerForContrastiveLearning.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=HF_CACHE).to(DEVICE)
print("models loaded")
#this crashes unless I run it first?

/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading models


/home/hice1/bzhou91/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BridgeTowerForContrastiveLearning were not initialized from the model checkpoint at BridgeTower/bridgetower-large-itm-mlm-itc and are newly initialized: ['logit_scale']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


models loaded


## Step 0 : download videos

In [3]:
import pandas as pd

df_questions = pd.read_csv('Handmade_questions.csv')
print(df_questions.keys())
df_questions = df_questions.dropna(subset=['Question']) #drop any rows that aren't filled yet
df_questions

Index(['VideoID', 'Question', 'Answer1 (corr)', 'Answer2', 'Answer3',
       'Answer4', 'youtubeURL', 'Unnamed: 7', 'Unnamed: 8',
       'https://www.youtube.com/watch?v='],
      dtype='object')


/scratch/1034961/ipykernel_1844531/4207397511.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,VideoID,Question,Answer1 (corr),Answer2,Answer3,Answer4,youtubeURL,Unnamed: 7,Unnamed: 8,https://www.youtube.com/watch?v=
0,zwbCD3CUFIs,What color is the spatula on the cutting board?,Green,Red,Blue,Black,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
1,zwbCD3CUFIs,What green ingredient is added to the pan ?,Spring onions,Green pepper,Green beans,Cucumber,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
2,zwbCD3CUFIs,What is the purpose of the large blue object o...,To provide heat to cook,It is a trash can,It is a pan,It's a decoration,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
3,zwbCD3CUFIs,How many prawns are cooking in the pan?,approximately 25,About 4 big prawns,approximately 80 prawns,there are no prawns,https://www.youtube.com/watch?v=zwbCD3CUFIs,NaN,NaN,NaN
5,XzgR14o6b80,What is the color if the liquid the two steaks...,Yellow-brown,Red,Green,Transparent,https://www.youtube.com/watch?v=XzgR14o6b80,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
73,i8BjqgemuXo,What distinguishes this dish?,Honey ham and parma ham,Bread type,Replace with turkey,Serve extra cold,https://www.youtube.com/watch?v=i8BjqgemuXo,NaN,NaN,NaN
75,g1jFJYtQ9_s,What flour is used traditionally for this dish?,Whole wheat flour,Cornmeal,Corn flour,Rice flour,https://www.youtube.com/watch?v=g1jFJYtQ9_s,NaN,NaN,NaN
76,g1jFJYtQ9_s,How long do you rest the dish prior to rolling?,30 minutes,20 minutes,10 minutes,45 minutes,https://www.youtube.com/watch?v=g1jFJYtQ9_s,NaN,NaN,NaN
77,g1jFJYtQ9_s,How to make the dish fluffy?,Open flame placement,Flattening with spatula,Add yeast to leaven,Freeze prior to cooking,https://www.youtube.com/watch?v=g1jFJYtQ9_s,NaN,NaN,NaN


In [4]:
ALL_URLS = df_questions['VideoID'].unique().tolist()

In [5]:
#################################################################################################################
WANT_TO_RUN_DL = True
#################################################################################################################



from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

base_path = './storage'

if WANT_TO_RUN_DL:

    for URL in ALL_URLS:

        os.makedirs(f"{base_path}/{URL}", exist_ok=True)
        url = f"https://www.youtube.com/watch?v={URL}"

        yt = YouTube(url, on_progress_callback = on_progress)
        print(yt.title)
        ys = yt.streams.get_highest_resolution()
        ys.download(f"{base_path}/{URL}", filename='video.mp4')

        caption = yt.captions.get_by_language_code('a.en')

        #save captions to ./
        caption.save_captions(f"{base_path}/{URL}/caption.srt")

Singapore-Style Curry Fried Rice Noodles | Asian Bites


/scratch/1034961/ipykernel_1844531/1902557335.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Experience Culinary Bliss with Gordon Ramsay Beef Tacos with Wasabi Mayo


/scratch/1034961/ipykernel_1844531/1902557335.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Naan Bread Recipe | How To Make Naan Bread at Home by Manjula


...

/scratch/1034961/ipykernel_1844531/1902557335.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Croque Monsieur | John Quilter


/scratch/1034961/ipykernel_1844531/1902557335.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Chapati recipe - unleavened bread


/scratch/1034961/ipykernel_1844531/1902557335.py:25: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


## Step 1 : Generate the CSV from input videos

In [6]:
import cv2
import os
import csv
import pysrt


csv_output = "image_transcript_mapping.csv"

def video_to_images_with_transcript(video_path, output_folder, srt_path, csv_output, frame_interval=1):
    """
    Extract video frames as images, and create csv in format of [index, image_path, transcript]

    video_path: Path to the input video file
    output_folder: Directory to save the image frames
    srt_path: Path to the SRT subtitle file
    csv_output: Path to the output CSV file mapping images to transcript text
    frame_interval: Interval of frames to save
    """
    video = cv2.VideoCapture(video_path)
    
    if not video.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    os.makedirs(output_folder, exist_ok=True)
    
    subs = pysrt.open(srt_path)
    
    fps = video.get(cv2.CAP_PROP_FPS)
    
    frame_count = 0
    saved_count = 0
    csv_data = []

    while True:
        success, frame = video.read()
        #mkdir f"{output_folder}/frames" with os
        os.makedirs(f"{output_folder}/frames", exist_ok=True)
        
        if not success:
            break

        if frame_count % frame_interval == 0:
            # Save the frame as image
            img_filename = f"{output_folder}/frames/frame_{frame_count:06d}.jpg"
            print(f"Saving frame {frame_count} to {img_filename}")
            cv2.imwrite(img_filename, frame)
            saved_count += 1

            # Map subtitle with timestamp

            timestamp = frame_count / fps

            transcript_text = ""
            for sub in subs:
                start_time = sub.start.ordinal / 1000
                end_time = sub.end.ordinal / 1000
                if abs(start_time - timestamp) < 11:
                    transcript_text += sub.text+" "
                    
            #make the img_filename the full path
            curr_path = os.path.join(os.getcwd(), img_filename)
            print(curr_path)

            if transcript_text == "":
                transcript_text = "No transcript available"
            csv_data.append([int(frame_count/frame_interval), curr_path, transcript_text,timestamp])

        frame_count += 1

    video.release()

    with open(f'{output_folder}/{csv_output}', mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['index', 'image_path', 'transcript_text','timestamp'])
        writer.writerows(csv_data)

    print(f"Saved {saved_count} frames from {video_path} to {output_folder}")
    print(f"Transcript mapping saved to {csv_output}")






In [7]:
import os
import glob
import json
import tqdm
import natsort
import random
import csv

from PIL import Image

import numpy as np

# import torch
from torch.utils.data import Dataset, DataLoader

import clip

from torchvision import models

# from config import config

#Original from LVNet

class loading_img(Dataset):
    def __init__(self, img_list):
        self.img_list = img_list

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        return preprocess(Image.open(self.img_list[idx]))


def update_csv(folder, out_frames, config_csv):
    with open(config_csv, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
    selected_frame_names = {os.path.basename(frame) for frame in out_frames}
    updated_rows = [row for row in rows if os.path.basename(row['image_path']) in selected_frame_names]
    for new_index, row in enumerate(updated_rows):
        row['index'] = new_index
    with open(config_csv, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
        writer.writeheader()
        writer.writerows(updated_rows)

# select frames
def select_frames(folder_list, csv_path, preprocess, resnet18_pretrained):
    divlam = 36
    num_picks = 1
    for folder in folder_list: 
        img_list = natsort.natsorted(glob.glob(f"{folder}/*.jpg"))
        img_feats = []    
        img_set = loading_img(img_list)
        img_loader = DataLoader(img_set, batch_size=64, shuffle=False, num_workers=0)

        for imgtensor in img_loader: img_feats.append(imgtensor)

        if len(img_feats) == 0:
            print("SKIPPED: Received No Frames")
            return None, None
        img_feats = torch.cat(img_feats, dim=0).to(device)

        with torch.no_grad():
            featuremap = resnet18_pretrained(img_feats)
            frame_num = featuremap.shape[0]

            dist_list = []
            for img_feat in featuremap: dist_list.append(torch.mean(torch.sqrt((featuremap-img_feat)**2), dim=-1))
            dist_list = torch.cat(dist_list).reshape(frame_num, frame_num)

            idx_list = [_ for _ in range(frame_num)]

            # print(dist_list)
            loop_idx = 0
            out_frames = []

            output_results = []
            while len(idx_list) > 5:
                dist_idx = idx_list.pop(0)

                data = dist_list[dist_idx, idx_list].softmax(dim=-1)
                mu, std = torch.mean(data), torch.std(data)
                pop_idx_list = torch.where(data < mu-std*(np.exp(1-loop_idx/divlam)))[0].detach().cpu().numpy()
                result = list(np.array(idx_list)[pop_idx_list])
                result.append(dist_idx)
                output_results.append(result)

#                 num_picks = config.num_picks
                if len(result) > num_picks:
                    idx_result_list = sorted(random.sample(result, num_picks)) 
                    img_list = np.array(img_list)
                    idx_result_list = np.array(idx_result_list)
                    out_frames.extend(img_list[idx_result_list])
                else:
                    idx_result_list = sorted(result)
                    img_list = np.array(img_list)
                    idx_result_list = np.array(idx_result_list)
                    out_frames.extend(img_list[idx_result_list])

                loop_idx += 1
                
                for pop_idx in reversed(pop_idx_list): idx_list.pop(pop_idx)
    
        update_csv(folder, out_frames, csv_path)
        for image in img_list:
            if image not in out_frames:
                os.remove(image)
    return out_frames, output_results


# Init


# output_results = []
# folder_list=glob.glob(config.img_folder)

# out_frames, output_result = select_frames(folder_list, config.csv, preprocess, resnet18_pretrained)
# output_results.append(output_result)
# print(output_results)    


In [8]:
# random.seed(10)
import torch
import clip

from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"

resnet18_pretrained = models.resnet18(pretrained=True).to(device)
resnet18_pretrained.fc = torch.nn.Identity()
resnet18_pretrained.avgpool = torch.nn.Identity()
resnet18_pretrained.eval()

model, preprocess = clip.load("ViT-B/32", device=device)

/home/hice1/bzhou91/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hice1/bzhou91/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

#################################################################################################################
WANT_TO_RUN_CSV_CREATION = True
INTERVAL_TIME = 30
#################################################################################################################

import os
import multiprocessing
from torchvision import models
import clip
import glob

def process_url(URL):
    """
    Process a single URL by extracting frames and generating CSV
    
    :param URL: The URL/path to process
    """
    video_path = f"{base_path}/{URL}/video.mp4"
    srt_path = f"{base_path}/{URL}/caption.srt"
    
    # Launch the script to extract frames and generate CSV
    video_to_images_with_transcript(video_path, f"{base_path}/{URL}", srt_path, csv_output, INTERVAL_TIME)
def main():
    # Determine the number of processes (4 as you specified, or use number of CPU cores)
    num_processes = 4
    
#     Use multiprocessing Pool to process URLs in parallel
    with multiprocessing.Pool(processes=num_processes) as pool:
        # Map the process_url function to ALL_URLS
        pool.map(process_url, ALL_URLS)     

if __name__ == '__main__':
    if WANT_TO_RUN_CSV_CREATION:
        main()

Saving frame 0 to ./storage/XzgR14o6b80/frames/frame_000000.jpgSaving frame 0 to ./storage/zwbCD3CUFIs/frames/frame_000000.jpgSaving frame 0 to ./storage/vow-kxTPatc/frames/frame_000000.jpgSaving frame 0 to ./storage/UUlfKJ9M_pQ/frames/frame_000000.jpg



/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000000.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/vow-kxTPatc/frames/frame_000000.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000000.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/UUlfKJ9M_pQ/frames/frame_000000.jpg



Saving frame 30 to ./storage/vow-kxTPatc/frames/frame_000030.jpgSaving frame 30 to ./storage/zwbCD3CUFIs/frames/frame_000030.jpg
Saving frame 30 to ./storage/UUlfKJ9M_pQ/frames/frame_000030.jpg

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_

Saving frame 360 to ./storage/vow-kxTPatc/frames/frame_000360.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/UUlfKJ9M_pQ/frames/frame_000330.jpg
Saving frame 330 to ./storage/zwbCD3CUFIs/frames/frame_000330.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/vow-kxTPatc/frames/frame_000360.jpg

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000330.jpgSaving frame 270 to ./storage/XzgR14o6b80/frames/frame_000270.jpg
Saving frame 360 to ./storage/UUlfKJ9M_pQ/frames/frame_000360.jpg

Saving frame 390 to ./storage/vow-kxTPatc/frames/frame_000390.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/UUlfKJ9M_pQ/frames/frame_000360.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000270.jpgSaving frame 360 to ./storage/zwbCD3CUFIs/frames/frame_000360.jpg
/

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/vow-kxTPatc/frames/frame_000720.jpg


/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000690.jpg
Saving frame 750 to ./storage/vow-kxTPatc/frames/frame_000750.jpgSaving frame 720 to ./storage/zwbCD3CUFIs/frames/frame_000720.jpg

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/vow-kxTPatc/frames/frame_000750.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000720.jpgSaving frame 510 to ./storage/XzgR14o6b80/frames/frame_000510.jpg

Saving frame 660 to ./storage/UUlfKJ9M_pQ/frames/frame_000660.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000510.jpg

Saving frame 780 to ./storage/vow-kxTPatc/frames/frame_000780.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custo

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000750.jpg
Saving frame 1080 to ./storage/zwbCD3CUFIs/frames/frame_001080.jpgSaving frame 930 to ./storage/UUlfKJ9M_pQ/frames/frame_000930.jpg

Saving frame 1140 to ./storage/vow-kxTPatc/frames/frame_001140.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_001080.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/UUlfKJ9M_pQ/frames/frame_000930.jpg

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/vow-kxTPatc/frames/frame_001140.jpg
Saving frame 780 to ./storage/XzgR14o6b80/frames/frame_000780.jpg
Saving frame 960 to ./storage/UUlfKJ9M_pQ/frames/frame_000960.jpgSaving frame 1170 to ./storage/vow-kxTPatc/frames/frame_001170.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_000780.jp


Saving frame 1410 to ./storage/zwbCD3CUFIs/frames/frame_001410.jpg
Saving frame 1020 to ./storage/XzgR14o6b80/frames/frame_001020.jpgSaving frame 1530 to ./storage/vow-kxTPatc/frames/frame_001530.jpg
Saving frame 1260 to ./storage/UUlfKJ9M_pQ/frames/frame_001260.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_001410.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/XzgR14o6b80/frames/frame_001020.jpg



/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/vow-kxTPatc/frames/frame_001530.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/UUlfKJ9M_pQ/frames/frame_001260.jpg

Saving frame 1560 to ./storage/vow-kxTPatc/frames/frame_001560.jpg
Saving frame 1440 to ./storage/zwbCD3CUFIs/frames/frame_001440.jpg
Saving frame 1050 to ./storage/XzgR14o6b80/frames/frame_001050.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/noteboo

...

Saving frame 8100 to ./storage/i8BjqgemuXo/frames/frame_008100.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/i8BjqgemuXo/frames/frame_008100.jpg
Saving frame 6660 to ./storage/g1jFJYtQ9_s/frames/frame_006660.jpg
Saving frame 8130 to ./storage/i8BjqgemuXo/frames/frame_008130.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_006660.jpg

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/i8BjqgemuXo/frames/frame_008130.jpg
Saving frame 8160 to ./storage/i8BjqgemuXo/frames/frame_008160.jpgSaving frame 6690 to ./storage/g1jFJYtQ9_s/frames/frame_006690.jpg

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/i8BjqgemuXo/frames/frame_008160.jpg/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_006690.jpg

Saving frame 8190 to ./storage/i8BjqgemuXo/frames/frame_0081

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_007770.jpg
Saving frame 7800 to ./storage/g1jFJYtQ9_s/frames/frame_007800.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_007800.jpg
Saving frame 7830 to ./storage/g1jFJYtQ9_s/frames/frame_007830.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_007830.jpg
Saving frame 7860 to ./storage/g1jFJYtQ9_s/frames/frame_007860.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_007860.jpg
Saving frame 7890 to ./storage/g1jFJYtQ9_s/frames/frame_007890.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_007890.jpg
Saving frame 7920 to ./storage/g1jFJYtQ9_s/frames/frame_007920.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_cu

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_009060.jpg
Saving frame 9090 to ./storage/g1jFJYtQ9_s/frames/frame_009090.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_009090.jpg
Saving frame 9120 to ./storage/g1jFJYtQ9_s/frames/frame_009120.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_009120.jpg
Saving frame 9150 to ./storage/g1jFJYtQ9_s/frames/frame_009150.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_009150.jpg
Saving frame 9180 to ./storage/g1jFJYtQ9_s/frames/frame_009180.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_009180.jpg
Saving frame 9210 to ./storage/g1jFJYtQ9_s/frames/frame_009210.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_cu

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_010350.jpg
Saving frame 10380 to ./storage/g1jFJYtQ9_s/frames/frame_010380.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_010380.jpg
Saving frame 10410 to ./storage/g1jFJYtQ9_s/frames/frame_010410.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_010410.jpg
Saving frame 10440 to ./storage/g1jFJYtQ9_s/frames/frame_010440.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_010440.jpg
Saving frame 10470 to ./storage/g1jFJYtQ9_s/frames/frame_010470.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_010470.jpg
Saving frame 10500 to ./storage/g1jFJYtQ9_s/frames/frame_010500.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeli

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011640.jpg
Saving frame 11670 to ./storage/g1jFJYtQ9_s/frames/frame_011670.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011670.jpg
Saving frame 11700 to ./storage/g1jFJYtQ9_s/frames/frame_011700.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011700.jpg
Saving frame 11730 to ./storage/g1jFJYtQ9_s/frames/frame_011730.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011730.jpg
Saving frame 11760 to ./storage/g1jFJYtQ9_s/frames/frame_011760.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011760.jpg
Saving frame 11790 to ./storage/g1jFJYtQ9_s/frames/frame_011790.jpg
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeli

In [10]:

#################################################################################################################
WANT_TO_RUN_CSV_CREATION_2 = True
#################################################################################################################

import os
import multiprocessing
from torchvision import models
import clip
import glob

def process_url_2(URL):
    """
    Process a single URL by extracting frames and generating CSV
    
    :param URL: The URL/path to process
    """
    video_path = f"{base_path}/{URL}/video.mp4"
    srt_path = f"{base_path}/{URL}/caption.srt"
    
    folder_list=glob.glob(f"{base_path}/{URL}/frames")
    out_frames, output_result = select_frames(folder_list, f"{base_path}/{URL}/image_transcript_mapping.csv", preprocess, resnet18_pretrained)    

In [11]:
#TSC Sampling
for url in ALL_URLS:
    process_url_2(url)
print("completed")

completed


In [12]:
# with torch.no_grad():
#     torch.cuda.empty_cache()

## Step 2 : Generate FAISS embeddings from the CSVs

In [13]:

#################################################################################################################
WANT_TO_CREATE_FAISS = True
#Takes about 15 sec per video on a good GPU (depends also how many frames per vid, this is for interval 60)
DEVICE = "cuda:0" #only runs fast on GPU

HF_CACHE = "/home/hice1/bzhou91/scratch/cache"
#################################################################################################################


import json
import os
import numpy as np
from numpy.linalg import norm
import cv2
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from termcolor import cprint
import faiss
import pandas as pd

print("loading models")
# BT_processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=HF_CACHE) #.to(DEVICE)
# BT_model = BridgeTowerForContrastiveLearning.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc",cache_dir=HF_CACHE).to(DEVICE)
print("models loaded")

def encode_single_image_text(image_path, text):
    print(image_path)
    print(text)
    image = Image.open(image_path)
    encoding = BT_processor(image, text, return_tensors="pt").to(DEVICE)
    outputs = BT_model(**encoding)
    return outputs['cross_embeds'].cpu().detach().numpy()




if WANT_TO_CREATE_FAISS:
    for URL in ALL_URLS:
        input_folder = f"{base_path}/{URL}"
        
        
            #load ./paired_image_data.csv
        df_mapping = pd.read_csv(f'{input_folder}/image_transcript_mapping.csv')
        #rows :             index,image_path,transcript_text,timestamp

        #create empty FAISS index
        

        # Create embeddings for each row
        embeddings = np.zeros((len(df_mapping), 512))
        for _, row in tqdm(df_mapping.iterrows()):


            embedding = encode_single_image_text(row['image_path'], row['transcript_text'])
            #reshape to (512,)
            embedding = embedding.reshape(512)
            embeddings[row['index'],:] = embedding

            print(embedding.shape)

        # Initialize FAISS index
        dimension = 512  # dimension of your embeddings
        index = faiss.IndexFlatL2(dimension)

        # Add embeddings to the FAISS index
        embeddings = np.array(embeddings, dtype=np.float32)
        index.add(embeddings)

        # Create a mapping between FAISS ids and dataframe indices
        id_to_index = {i: idx for i, idx in enumerate(df_mapping.index)}

        # Save the FAISS index
        faiss.write_index(index, f"{input_folder}/faiss_database.bin")
        print('saved faiss index')

/home/hice1/bzhou91/.local/lib/python3.10/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


loading models
models loaded


0it [00:00, ?it/s]

/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/zwbCD3CUFIs/frames/frame_000030.jpg
hey guys I'm Amy and I'm Julie we are the Duncan sisters welcome to the scoffs kitchen today in our Asian bite series we are doing a Singapore fried rice noodle with curry is you can see it's this bright yellow sort of concoction 


3it [00:00,  3.95it/s]

...

(512,)
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011310.jpg
when I see brown spots on both sides it is ready to come out and these are now 
(512,)
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_011580.jpg
when I see brown spots on both sides it is ready to come out and these are now ready to serve I made six chapatis three full cars on the open flame and three 


44it [00:03, 12.27it/s]

(512,)
/storage/ice1/4/5/bzhou91/LLaVA_RAG_PoC/notebook_pipeline_custom_questions/./storage/g1jFJYtQ9_s/frames/frame_012150.jpg
full cars on the open flame and three chapathis on the griddle with ghee you can serve chapati as a part of a main meal with curries and are do try it sometime it's easy good food that was easy cooking thank you for watching 


45it [00:03, 12.17it/s]

(512,)
saved faiss index


## Step 3 : Chat with FAISS Db as RAG

In [14]:
import faiss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
from termcolor import cprint
import pandas as pd


def encode_text_query(text):
    global index
    global df
    #we suppose BT_processor and BT_model are loaded from cell above
    #create random Image 32x32
    image = Image.fromarray(np.random.randint(0,255,(32,32,3),dtype=np.uint8))
    encoding = BT_processor(image, text, return_tensors="pt").to(DEVICE)
    outputs = BT_model(**encoding)
    #reshape to (512,)

    return outputs['text_embeds'].cpu().detach().numpy().reshape(-1)


def get_n_closest_index(embedding,n):
    global index
    global df
    D, I = index.search(np.array([embedding]), n)
    #I[0] has the indexes of the n closest images
    return I[0]

def set_index(folder_path):
    global index
    global df
    index = faiss.read_index(folder_path+'/faiss_database.bin')
    df = pd.read_csv(folder_path+'/image_transcript_mapping.csv')



In [15]:
def return_image_and_enhanced_query(question,answer_a,answer_b,answer_c,answer_d,folder_path):
    global index
    global df
    set_index(folder_path)
    text_embedding = encode_text_query(question)
    
    #return query with 1 closest image/text pair to enhance the query

    closest_index = get_n_closest_index(text_embedding,1)
    print(f"Closest index is {closest_index[0]} for query '{question}'")
    return create_enhanced_conversation(df.iloc[closest_index[0]]['transcript_text'],question,Image.open(df.iloc[closest_index[0]]['image_path']),answer_a,answer_b,answer_c,answer_d)


def debug_top3_frames(question,answer_a,answer_b,answer_c,answer_d,folder_path):
    global index
    global df
    set_index(folder_path)
    text_embedding = encode_text_query(question)
    
    
    


    closest_index = get_n_closest_index(text_embedding,3)
    top1_text = df.iloc[closest_index[0]]['transcript_text']
    return closest_index[0],closest_index[1],closest_index[2],top1_text
    

def create_enhanced_conversation(textt,question,image,answer_a,answer_b,answer_c,answer_d):
    global index
    global df
    
    return [{
        "role": "user",
        "content": [
            {"type": "text", "text" : f"""Please answer this question : 'What is 2+2'
            This transcript can also help you answer : 'Addition is a mathematical commutative operation that' .
            It is possible that the image helps you answer the question, but it is possible that it does not
            Answer A: 4
            Answer B: 6
            Answer C: 9
            Answer D: 12"""}
        ],
    },
    {
        "role": "assistant",
        "content": [
   
            {"type": "text", "text" : "A"}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text" : f"""Please answer this question : '{question}'
            This transcript can also help you answer : '{textt}' .
            It is possible that the image helps you answer the question, but it is possible that it does not
            Answer A: {answer_a}
            Answer B: {answer_b}
            Answer C: {answer_c}
            Answer D: {answer_d}"""}
        ],
    }],image

In [16]:
#This script loads LLaVA so it's on a big GPU
import json
import os
import numpy as np
from numpy.linalg import norm
import cv2
import pandas as pd
from tqdm import tqdm
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from termcolor import cprint


########################## LOADING LLAVA ##########################


print("beginning to load LLaVA")

llava_processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", device=DEVICE,cache_dir=HF_CACHE)


from transformers import LlavaNextForConditionalGeneration, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,  # Threshold for weight offloading
    llm_int8_skip_modules=["lm_head"],  # Prevent quantization of specific layers
    llm_int8_has_fp16_weights=True  # Mixed precision with fp16 weights
)

llava_model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", quantization_config=quantization_config, device_map=DEVICE,cache_dir=HF_CACHE)


beginning to load LLaVA


Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Unused kwargs: ['llm_int8_has_fp16_weights']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 4/4 [01:32<00:00, 23.10s/it]


In [17]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# Initialize an empty list to store results
results = []

total_correct = 0
for index, row in df_questions.iterrows():
    URL = row['VideoID']
    question = row['Question']
    correct_answer = row['Answer1 (corr)']
    wrong_answer1 = row['Answer2']
    wrong_answer2 = row['Answer3']
    wrong_answer3 = row['Answer4']
    
    # Shuffle and assign answer_a, b, c, d
    answers = [correct_answer, wrong_answer1, wrong_answer2, wrong_answer3]
    np.random.shuffle(answers)
    answer_a = answers[0]
    answer_b = answers[1]
    answer_c = answers[2]
    answer_d = answers[3]
    
    folder_path = f"{base_path}/{URL}"
    
    conv, image = return_image_and_enhanced_query(question, answer_a, answer_b, answer_c, answer_d, folder_path)
    top1, top2, top3,top1text = debug_top3_frames(question, answer_a, answer_b, answer_c, answer_d, folder_path)
    
    # Paths for top frames
    path_top1 = f"{folder_path}/frames/frame_{(INTERVAL_TIME*top1):06d}.jpg"
    path_top2 = f"{folder_path}/frames/frame_{(INTERVAL_TIME*top2):06d}.jpg"
    path_top3 = f"{folder_path}/frames/frame_{(INTERVAL_TIME*top3):06d}.jpg"
    
    prompt = llava_processor.apply_chat_template(conv, add_generation_prompt=True)
    inputs = llava_processor(image, prompt, return_tensors="pt").to(DEVICE)
    
    with torch.no_grad():
        # Get the model's output logits directly
        model_outputs = llava_model(**inputs)
        logits = model_outputs.logits[0, -1, :]
        
        # Apply softmax to get probabilities
        probabilities = F.softmax(logits, dim=0)
        
        answer_tokens = {
            'C': 334,
            'A': 330,
            'B': 365,
            'D': 384
        }
        
        # Check probabilities of known answer tokens
        answer_probs = {label: probabilities[token_id].item() 
                        for label, token_id in answer_tokens.items()}
        
        # Find the top answer among the known tokens
        top_answer = max(answer_probs, key=answer_probs.get)
        
        def top_answer_to_idx(character):
            if character == "A":
                return 0
            if character == "B":
                return 1
            if character == "C":
                return 2
            if character == "D":
                return 3
        
        top_answer_text = answers[top_answer_to_idx(top_answer)]
        
        # Determine if the selected answer is correct
        is_correct = (top_answer_text == correct_answer)
        if is_correct:
            total_correct += 1
        
        # Check if top token is not in known answers
        top_token_id = torch.argmax(probabilities).item()
        not_top_token = top_token_id not in answer_tokens.values()
        if not_top_token:
            top_token = llava_processor.tokenizer.convert_ids_to_tokens([top_token_id])[0]
        else:
            top_token = None
        
        # Prepare result dictionary
        result = {
            'VideoURL': URL,
            'Question': question,
            'Selected Answer': top_answer_text,
            'isSelectedAnswerCorrect': is_correct,
            'Choice A': answer_a,
            'Choice A Probability': answer_probs.get('A', None),
            'Choice B': answer_b,
            'Choice B Probability': answer_probs.get('B', None),
            'Choice C': answer_c,
            'Choice C Probability': answer_probs.get('C', None),
            'Choice D': answer_d,
            'Choice D Probability': answer_probs.get('D', None),
            'Correct Answer': correct_answer,
            'Not Top Token': not_top_token,
            'Top Token': top_token,
            'Chosen index for top frame': top1,
            'Second best index': top2,
            'Third best index': top3,
            'Path Top1': path_top1,
            'Path Top2': path_top2,
            'Path Top3': path_top3,
            'top 1 text' : top1text
        }
        
        results.append(result)
        
        print(f"Question: {question}")
        print(f"Selected Answer: {top_answer_text}")
        print(f"Correct: {is_correct}")
        print('----------------------------------------------------------------------')

# Create DataFrame from results
results_df = pd.DataFrame(results)

import time
t = time.time()
# Save to CSV
results_df.to_csv(f'{t}_model_answer_results.csv', index=False)

# Optional: Print total correct answers
print(f"Total Correct Answers: {total_correct}/{len(df_questions)}")
print(f"Accuracy: {total_correct/len(df_questions)*100:.2f}%")


Closest index is 23 for query 'What color is the spatula on the cutting board?'


/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/jinja2/lexer.py:652: DeprecationWarning: invalid escape sequence '\s'
  .decode("unicode-escape")


Question: What color is the spatula on the cutting board?
Selected Answer: Black
Correct: False
----------------------------------------------------------------------
Closest index is 10 for query 'What green ingredient is added to the pan ?'
Question: What green ingredient is added to the pan ?
Selected Answer: Green pepper
Correct: False
----------------------------------------------------------------------
Closest index is 12 for query 'What is the purpose of the large blue object on the table ?'
Question: What is the purpose of the large blue object on the table ?
Selected Answer: To provide heat to cook
Correct: True
----------------------------------------------------------------------
Closest index is 15 for query 'How many prawns are cooking in the pan?'
Question: How many prawns are cooking in the pan?
Selected Answer: About 4 big prawns
Correct: False
----------------------------------------------------------------------
Closest index is 35 for query 'What is the color if the

Question: What is used to garnish the dish?
Selected Answer: green onion
Correct: False
----------------------------------------------------------------------
Closest index is 15 for query 'Which utensil is used to cook the dish?'
Question: Which utensil is used to cook the dish?
Selected Answer: Ladle
Correct: True
----------------------------------------------------------------------
Closest index is 1 for query 'What is the name of the restaurant?'
Question: What is the name of the restaurant?
Selected Answer: Cha Ya Asian Bistro
Correct: True
----------------------------------------------------------------------
Closest index is 3 for query 'What meat is used in the recipe?'
Question: What meat is used in the recipe?
Selected Answer: Veal
Correct: True
----------------------------------------------------------------------
Closest index is 35 for query 'How does the chef add acid to the dish?'
Question: How does the chef add acid to the dish?
Selected Answer: Vinegar
Correct: False


In [18]:
from datetime import datetime
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2024-12-11 00:19:40


In [19]:
!nvidia-smi


Wed Dec 11 00:19:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           On  |   00000000:3B:00.0 Off |                    0 |
| N/A   74C    P0             58W /  250W |   20144MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
